In [7]:
import numpy as np
import matplotlib.pyplot as plt

import pywt
import pywt.data
import sys

if '..' not in sys.path:
    sys.path.append('..')

from stego.transform import blocking, dwt

In [8]:
original = pywt.data.camera()

In [9]:
c = pywt.dwt2(original, 'haar')
img = pywt.idwt2(c, 'haar')
print(f"{original=} {img=}")
print(f"{np.allclose(original, img)=}")

original=array([[200, 200, 200, ..., 189, 190, 190],
       [200, 199, 199, ..., 190, 190, 190],
       [199, 199, 199, ..., 190, 190, 190],
       ...,
       [ 25,  25,  27, ..., 139, 122, 147],
       [ 25,  25,  26, ..., 158, 141, 168],
       [ 25,  25,  27, ..., 151, 152, 149]], dtype=uint8) img=array([[200., 200., 200., ..., 189., 190., 190.],
       [200., 199., 199., ..., 190., 190., 190.],
       [199., 199., 199., ..., 190., 190., 190.],
       ...,
       [ 25.,  25.,  27., ..., 139., 122., 147.],
       [ 25.,  25.,  26., ..., 158., 141., 168.],
       [ 25.,  25.,  27., ..., 151., 152., 149.]])
np.allclose(original, img)=True


In [10]:
iwt = dwt.Iwt('haar', level=3)
original = pywt.data.camera()

iwt.forward(original)
img = iwt.inverse()
print(f"{original=} {img=}")
print(f"{np.allclose(original, img)=}")

original=array([[200, 200, 200, ..., 189, 190, 190],
       [200, 199, 199, ..., 190, 190, 190],
       [199, 199, 199, ..., 190, 190, 190],
       ...,
       [ 25,  25,  27, ..., 139, 122, 147],
       [ 25,  25,  26, ..., 158, 141, 168],
       [ 25,  25,  27, ..., 151, 152, 149]], dtype=uint8) img=array([[200., 200., 200., ..., 189., 190., 190.],
       [200., 199., 199., ..., 190., 190., 190.],
       [199., 199., 199., ..., 190., 190., 190.],
       ...,
       [ 25.,  25.,  27., ..., 139., 122., 147.],
       [ 25.,  25.,  26., ..., 158., 141., 168.],
       [ 25.,  25.,  27., ..., 151., 152., 149.]])
np.allclose(original, img)=True


In [11]:
import cv2

cv2.imwrite("comp2.png", original)
img = cv2.imread("comp2.png")
img, g, b = cv2.split(img)
print(f"{original=} {img=}")
print(f"{np.allclose(original, img)=}")

original=array([[200, 200, 200, ..., 189, 190, 190],
       [200, 199, 199, ..., 190, 190, 190],
       [199, 199, 199, ..., 190, 190, 190],
       ...,
       [ 25,  25,  27, ..., 139, 122, 147],
       [ 25,  25,  26, ..., 158, 141, 168],
       [ 25,  25,  27, ..., 151, 152, 149]], dtype=uint8) img=array([[200, 200, 200, ..., 189, 190, 190],
       [200, 199, 199, ..., 190, 190, 190],
       [199, 199, 199, ..., 190, 190, 190],
       ...,
       [ 25,  25,  27, ..., 139, 122, 147],
       [ 25,  25,  26, ..., 158, 141, 168],
       [ 25,  25,  27, ..., 151, 152, 149]], dtype=uint8)
np.allclose(original, img)=True


In [12]:
from stego.transform import blocking, dwt
original = pywt.data.camera()
blocks = blocking.divide_image(original)
img = blocking.stack_image(blocks)

print(f"{np.allclose(original, img)=}")

np.allclose(original, img)=True
